In [18]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso,LassoCV
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Section1 load data

In [1]:
cd "C:\Users\Administrator\Desktop\resting_state_eeg\children\resting EEG_machine learning\data\数据整理\"

C:\Users\Administrator\Desktop\resting_state_eeg\children\resting EEG_machine learning\data\数据整理


In [5]:
data_thickavg = pd.read_excel('DKTatlas40_analysis.xlsx', 'thickavg')

In [6]:
thick = data_thickavg.columns[4:]
X = data_thickavg[thick]
y = data_thickavg['Age']

# Section2 Optimize lasso model via GridSearch

In [7]:
X_train = X.head(301)
y_train = y.head(301)
X_test = X.tail(100)
y_test = y.tail(100)

In [32]:
pipe_lasso=make_pipeline(StandardScaler(),Lasso(normalize = "True"))

In [37]:
param_range_alpha= np.logspace(-5,2,200)
param_grid=[{'lasso__alpha':param_range_alpha}]


gs=GridSearchCV(estimator=pipe_lasso  ,
                param_grid=param_grid,
                scoring='accuracy',
                cv=10)
scores=cross_val_score(gs,X_train,y_train, scoring='neg_mean_absolute_error',cv=10)


gs.fit(X_train,y_train)
y_pred = gs.predict(X_test)
a = np.vstack((y_test, y_pred))

# Section3 Output

In [38]:
#optimal parameters 
print(gs.best_params_)

#Mean cross-validation score of the best estimator
print("MAE in Train Phase: %.3f" % gs.best_score_)

#correlation between y_predict and y_test
print(np.corrcoef(a))

print("CV Accuracy in Train Phase: %.3f +/- %.3f" % (np.mean(scores),np.std(scores)))
print(scores)
#print(gs.cv_results_)
print(np.mean(abs(y_pred - y_test)))

{'lasso__alpha': 0.002899422853882875}
MAE in Train Phase: -0.969
[[1.         0.40868548]
 [0.40868548 1.        ]]
CV Accuracy in Train Phase: -0.972 +/- 0.160
[-0.941866   -0.94158138 -0.78687193 -0.93735242 -1.13119272 -0.73067594
 -1.03056169 -0.84451166 -1.28848747 -1.09057232]
1.045374770821503


# Section4 Model optimization

In [39]:
#构造不同的lambda值
Lambdas=np.logspace(-5,2,200)
#设置交叉验证的参数，使用均方误差评估
lasso_cv=LassoCV(alphas=Lambdas,normalize=True,cv=10,max_iter=10000)
lasso_cv.fit(X_train,y_train)

#基于最佳lambda值建模
lasso=Lasso(alpha=lasso_cv.alpha_,normalize=True,max_iter=10000)
lasso.fit(X_train,y_train)
#打印回归系数
print(pd.Series(index=['Intercept']+X_train.columns.tolist(),
                data=[lasso.intercept_]+lasso.coef_.tolist()))
#模型评估
y_pred=lasso.predict(X_test)
#MAE
MAE = np.mean(abs(y_pred - y_test))
print(MAE)
#测试集预测年龄和实际年龄的相关
a = np.vstack((y_test, y_pred))
print(np.corrcoef(a))

Intercept                            16.335227
caudalanteriorcingulate_Lthickavg    -1.130277
caudalmiddlefrontal_Lthickavg         1.745858
cuneus_Lthickavg                     -1.156695
entorhinal_Lthickavg                  0.151858
                                       ...    
superiorparietal_Rthickavg           -0.734777
superiortemporal_Rthickavg            0.152422
supramarginal_Rthickavg              -0.000000
transversetemporal_Rthickavg         -0.444682
insula_Rthickavg                      0.645619
Length: 63, dtype: float64
1.0453747708215035
[[1.         0.40868548]
 [0.40868548 1.        ]]


In [48]:
lasso_cv.mse_path_

array([[1.91455998, 1.69745965, 1.61719235, ..., 1.90611612, 2.97813238,
        2.01370724],
       [1.91455998, 1.69745965, 1.61719235, ..., 1.90611612, 2.97813238,
        2.01370724],
       [1.91455998, 1.69745965, 1.61719235, ..., 1.90611612, 2.97813238,
        2.01370724],
       ...,
       [1.40279884, 1.3601215 , 1.01845506, ..., 1.05063945, 2.37458681,
        2.26782796],
       [1.40295323, 1.36033877, 1.01860168, ..., 1.05059016, 2.37474128,
        2.26815744],
       [1.40309573, 1.36053936, 1.01865701, ..., 1.05054656, 2.37488147,
        2.26846156]])